[도전과제] 

 아래는 백화점 고객의 1년 간 구매 데이터이다.
 
(가) 제공 데이터 목록  
① y_train.csv : 고객의 성별 데이터 (학습용), CSV 형식의 파일  
② X_train.csv, X_test.csv : 고객의 상품구매 속성 (학습용 및 평가용), CSV 형식의 파일  
(나) 데이터 형식 및 내용  
① y_train.csv (3,500명 데이터)  
② X_train.csv (3,500명 데이터), X_test.csv (2,482명 데이터)  

고객 3,500명에 대한 학습용 데이터(y_train.csv, X_train.csv)를 이용하여 성별예측 모형을 만든 
후, 이를 평가용 데이터(X_test.csv)에 적용하여 얻은 2,482명 고객의 성별 예측값(남자일 확률)을 
다음과 같은 형식의 CSV 파일로 생성하시오.(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 
채점)  

<제출형식>
 custid,gender  
 3500,0.267  
 3501,0.578  
 3502,0.885  
 ․ ․ ․ 
 
 <유의사항>  
성능이 우수한 예측모형을 구축하기 위해서는 적절한 데이터 전처리, Feature Engineering, 분류 
알고리즘 사용, 초매개변수 최적화, 모형 앙상블 등이 수반되어야 한다

 X_train.csv,y_train.csv 로 train,test 으로 나눠서 평가 0.7정도가 나오면 상당히 높은 편 여성이 gender 0

In [104]:
import pandas as pd
X_df = pd.read_csv('./dataset/X_train.csv',encoding='CP949')
y_df = pd.read_csv('./dataset/y_train.csv',encoding='CP949')

In [105]:
display(X_df.head())
display(y_df.head())

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1
3,3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16
4,4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85


,cust_id,gender
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0


In [106]:
# cust_id는 고유 번호이므로 삭제
X = X_df.drop('cust_id',axis=1)
y = y_df.drop('cust_id',axis=1)

In [107]:
# gender 와의 상관성을 찾기 위하여 gender를 합친 데이터 프레임 생성
Xy = pd.concat([X,y],axis =1)
Xy.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기,gender
0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17,0
1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1,0
2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1,1
3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16,1
4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85,0


In [108]:
Xy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   총구매액     3500 non-null   int64  
 1   최대구매액    3500 non-null   int64  
 2   환불금액     1205 non-null   float64
 3   주구매상품    3500 non-null   object 
 4   주구매지점    3500 non-null   object 
 5   내점일수     3500 non-null   int64  
 6   내점당구매건수  3500 non-null   float64
 7   주말방문비율   3500 non-null   float64
 8   구매주기     3500 non-null   int64  
 9   gender   3500 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 273.6+ KB


In [109]:
Xy.corr()

,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기,gender
총구매액,1.000000,0.700080,0.419734,0.659084,0.090022,0.014396,-0.212944,-0.150141
최대구매액,0.700080,1.000000,0.410562,0.374147,0.018980,0.022277,-0.115837,-0.114323
환불금액,0.419734,0.410562,1.000000,0.270290,-0.063114,-0.062397,-0.211125,-0.078025
내점일수,0.659084,0.374147,0.270290,1.000000,0.225264,-0.010325,-0.293200,-0.155000
내점당구매건수,0.090022,0.018980,-0.063114,0.225264,1.000000,0.007659,-0.091151,-0.043917
주말방문비율,0.014396,0.022277,-0.062397,-0.010325,0.007659,1.000000,0.003372,0.073598
구매주기,-0.212944,-0.115837,-0.211125,-0.293200,-0.091151,0.003372,1.000000,0.041450
gender,-0.150141,-0.114323,-0.078025,-0.155000,-0.043917,0.073598,0.041450,1.000000


In [110]:
# 남여 비율 파악
Xy.gender.value_counts()

0    2184
1    1316
Name: gender, dtype: int64

In [111]:
# 주구매 상품과의 연관성 파악
Xy[['주구매상품','gender']].groupby('주구매상품').mean()

,gender
주구매상품,
가공식품,0.419414
가구,0.500000
건강식품,0.531915
골프,0.536585
구두,0.518519
기타,0.369748
남성 캐주얼,0.763636
남성 트랜디,0.500000
남성정장,0.818182


In [112]:
# 라벨 인코딩으로 주구매상품 분류
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['주구매상품'] = le.fit_transform(X['주구매상품'])
X.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,68282840,11264000,6860000.0,5,강남점,19,3.894737,0.527027,17
1,2136000,2136000,300000.0,21,잠실점,2,1.500000,0.000000,1
2,3197000,1639000,NaN,6,관악점,2,2.000000,0.000000,1
3,16077620,4935000,NaN,5,광주점,18,2.444444,0.318182,16
4,29050000,24000000,NaN,15,본 점,2,1.500000,0.000000,85


In [113]:
# 주구매 상품과의 연관성 파악
Xy[['주구매지점','gender']].groupby('주구매지점').mean()

,gender
주구매지점,
강남점,0.255172
관악점,0.413043
광주점,0.403509
노원점,0.311111
대구점,0.714286
대전점,0.357143
동래점,0.346939
미아점,0.434783
본 점,0.402971


In [114]:
# 주구매지점 역시 라벨인코딩으로 변환
X['주구매지점'] = le.fit_transform(X['주구매지점'])
X.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,68282840,11264000,6860000.0,5,0,19,3.894737,0.527027,17
1,2136000,2136000,300000.0,21,19,2,1.500000,0.000000,1
2,3197000,1639000,NaN,6,1,2,2.000000,0.000000,1
3,16077620,4935000,NaN,5,2,18,2.444444,0.318182,16
4,29050000,24000000,NaN,15,8,2,1.500000,0.000000,85


In [115]:
# 환불금액의 데이터 탐색
Xy.환불금액.describe()

count    1.205000e+03
mean     2.407822e+07
std      4.746453e+07
min      5.600000e+03
25%      2.259000e+06
50%      7.392000e+06
75%      2.412000e+07
max      5.637530e+08
Name: 환불금액, dtype: float64

In [116]:
# 환불 여부와 성별 상관관계 파악
import numpy as np
Xy.환불금액.fillna(0,inplace = True)
Xy['환불금액'] = np.where(Xy.환불금액 == 0 ,0,1)
Xy.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기,gender
0,68282840,11264000,1,기타,강남점,19,3.894737,0.527027,17,0
1,2136000,2136000,1,스포츠,잠실점,2,1.500000,0.000000,1,0
2,3197000,1639000,0,남성 캐주얼,관악점,2,2.000000,0.000000,1,1
3,16077620,4935000,0,기타,광주점,18,2.444444,0.318182,16,1
4,29050000,24000000,0,보석,본 점,2,1.500000,0.000000,85,0


In [117]:
Xy[['환불금액','gender']].groupby('환불금액').mean()

,gender
환불금액,
0,0.447059
1,0.240664


In [118]:
# X에 Xy의 데이터로 대체
X['환불금액']=Xy['환불금액']
X.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,68282840,11264000,1,5,0,19,3.894737,0.527027,17
1,2136000,2136000,1,21,19,2,1.500000,0.000000,1
2,3197000,1639000,0,6,1,2,2.000000,0.000000,1
3,16077620,4935000,0,5,2,18,2.444444,0.318182,16
4,29050000,24000000,0,15,8,2,1.500000,0.000000,85


In [119]:
# 최대 구매액 범주화
X.최대구매액.describe()
def get_catmax(X):
    cat = ''                #여기에 저장됨
    if X < 2.875000e+06 : cat = 1
    elif X < 9.837000e+06: cat = 2
    elif X < 2.296250e+07: cat = 3
    else:
        cat = 4
    return cat
Xy['최대구매액'] = Xy['최대구매액'].apply(lambda X : get_catmax(X))
Xy.head()


,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기,gender
0,68282840,3,1,기타,강남점,19,3.894737,0.527027,17,0
1,2136000,1,1,스포츠,잠실점,2,1.500000,0.000000,1,0
2,3197000,1,0,남성 캐주얼,관악점,2,2.000000,0.000000,1,1
3,16077620,2,0,기타,광주점,18,2.444444,0.318182,16,1
4,29050000,4,0,보석,본 점,2,1.500000,0.000000,85,0


In [120]:
# 나누어진 범주로 성별과의 연관성 파악
Xy[['최대구매액','gender']].groupby('최대구매액').mean()

,gender
최대구매액,
1,0.522883
2,0.372146
3,0.342857
4,0.266286


In [121]:
# X에도 범주 적용
X['최대구매액'] = X['최대구매액'].apply(lambda X : get_catmax(X))
X.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,68282840,3,1,5,0,19,3.894737,0.527027,17
1,2136000,1,1,21,19,2,1.500000,0.000000,1
2,3197000,1,0,6,1,2,2.000000,0.000000,1
3,16077620,2,0,5,2,18,2.444444,0.318182,16
4,29050000,4,0,15,8,2,1.500000,0.000000,85


In [122]:
Xy.총구매액.describe()

count    3.500000e+03
mean     9.191925e+07
std      1.635065e+08
min     -5.242152e+07
25%      4.747050e+06
50%      2.822270e+07
75%      1.065079e+08
max      2.323180e+09
Name: 총구매액, dtype: float64

In [123]:
# 총 구매액 범주화
def get_catall(X):
    cat = ''                #여기에 저장됨
    if X < 4.747050e+06 : cat = 1
    elif X < 2.822270e+07 : cat = 2
    elif X < 1.065079e+08 : cat = 3
    else:
        cat = 4
    return cat
Xy['총구매액'] = Xy['총구매액'].apply(lambda X : get_catall(X))
Xy.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기,gender
0,3,3,1,기타,강남점,19,3.894737,0.527027,17,0
1,1,1,1,스포츠,잠실점,2,1.500000,0.000000,1,0
2,1,1,0,남성 캐주얼,관악점,2,2.000000,0.000000,1,1
3,2,2,0,기타,광주점,18,2.444444,0.318182,16,1
4,3,4,0,보석,본 점,2,1.500000,0.000000,85,0


In [124]:
# 나누어진 범주로 성별과의 연관성 파악
Xy[['총구매액','gender']].groupby('총구매액').mean()

,gender
총구매액,
1,0.507429
2,0.427429
3,0.336000
4,0.233143


In [125]:
# X에도 범주 적용
X['총구매액'] = X['총구매액'].apply(lambda X : get_catall(X))
X.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,3,3,1,5,0,19,3.894737,0.527027,17
1,1,1,1,21,19,2,1.500000,0.000000,1
2,1,1,0,6,1,2,2.000000,0.000000,1
3,2,2,0,5,2,18,2.444444,0.318182,16
4,3,4,0,15,8,2,1.500000,0.000000,85


In [126]:
# 주말방문비율 탐색 : minmax인코딩이 이미 되어있는것으로 보이기에 그대로 둠
X.주말방문비율.describe()

count    3500.000000
mean        0.307246
std         0.289752
min         0.000000
25%         0.027291
50%         0.256410
75%         0.448980
max         1.000000
Name: 주말방문비율, dtype: float64

In [127]:
Xy.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기,gender
0,3,3,1,기타,강남점,19,3.894737,0.527027,17,0
1,1,1,1,스포츠,잠실점,2,1.500000,0.000000,1,0
2,1,1,0,남성 캐주얼,관악점,2,2.000000,0.000000,1,1
3,2,2,0,기타,광주점,18,2.444444,0.318182,16,1
4,3,4,0,보석,본 점,2,1.500000,0.000000,85,0
...,...,...,...,...,...,...,...,...,...,...
3495,1,2,0,골프,본 점,1,2.000000,1.000000,0,1
3496,3,2,1,시티웨어,부산본점,8,1.625000,0.461538,40,1
3497,1,1,0,주방용품,창원점,1,1.000000,0.000000,0,0
3498,1,1,0,화장품,본 점,2,1.000000,0.000000,39,0


In [158]:
X.내점당구매건수.describe()

count    3500.000000
mean        2.846000
std         1.925149
min         1.000000
25%         2.000000
50%         2.000000
75%         3.000000
max        22.000000
Name: 내점당구매건수, dtype: float64

In [128]:
# 내점당 구매건수 파악
X.내점당구매건수.describe()
Xy['내점당구매건수'] = round(Xy.내점당구매건수)
Xy[['내점당구매건수','gender']].groupby('내점당구매건수').mean()

,gender
내점당구매건수,
1.0,0.465798
2.0,0.378561
3.0,0.330532
4.0,0.332512
5.0,0.325581
6.0,0.400000
7.0,0.403509
8.0,0.448276
9.0,0.260870


In [129]:
# 반올림하기로 결정
X['내점당구매건수'] = round(X.내점당구매건수)

In [131]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,roc_auc_score,f1_score

def score_cal(y_test,y_pred=None,y_pred_proba=None):
    acc = accuracy_score(y_test,y_pred)
    pre = precision_score(y_test,y_pred)
    rec = recall_score(y_test,y_pred)
    f1 = f1_score(y_test,y_pred)    
    roc = roc_auc_score(y_test,y_pred_proba)
    print(f'accuracy : {round(acc,4)}, precision : {round(pre,4)}, recall : {round(rec,4)}, f1_score : {round(f1,4)}, roc_auc : {round(roc,4)}')

X_train ,X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=11)
lr = LogisticRegression()
rf = RandomForestClassifier()
dt = DecisionTreeClassifier()

lr.fit(X_train,y_train)
rf.fit(X_train,y_train)
dt.fit(X_train,y_train)

lr_pred = lr.predict(X_test)
lr_pred_proba = lr.predict_proba(X_test)[:,1]
rf_pred = rf.predict(X_test)
rf_pred_proba = rf.predict_proba(X_test)[:,1]
dt_pred = dt.predict(X_test)
dt_pred_proba = dt.predict_proba(X_test)[:,1]

In [141]:
print('lr 스코어')
score_cal(y_test,lr_pred,lr_pred_proba)

lr 스코어
accuracy : 0.6543, precision : 0.5704, recall : 0.295, f1_score : 0.3889, roc_auc : 0.6415


In [142]:
print('rf 스코어')
score_cal(y_test,rf_pred,rf_pred_proba)

rf 스코어
accuracy : 0.6286, precision : 0.5031, recall : 0.3103, f1_score : 0.3839, roc_auc : 0.6284


In [143]:
print('dt 스코어')
score_cal(y_test,dt_pred,dt_pred_proba)

dt 스코어
accuracy : 0.5943, precision : 0.4566, recall : 0.4636, f1_score : 0.4601, roc_auc : 0.5699


In [135]:
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

parameters = {'max_depth':[2,3,5,10],'min_samples_split':[2,3,5],'min_samples_leaf':[1,2,3]}
grid_rf = GridSearchCV(rf,param_grid=parameters,cv=3,refit=True)
grid_rf.fit(X_train,y_train)
print('rf 최적 파라미터 : ',grid_rf.best_params_)
grid_rf_pred = grid_rf.best_estimator_.predict(X_test)
grid_rf_pred_proba = grid_rf.best_estimator_.predict_proba(X_test)[:,1]

parameters = {'max_depth':[2,3,5,10],'min_samples_split':[2,3,5],'min_samples_leaf':[1,2,3]}
grid_dt = GridSearchCV(dt,param_grid=parameters,cv=3,refit=True)
grid_dt.fit(X_train,y_train)
print('dt 최적 파라미터 : ',grid_dt.best_params_)
grid_dt_pred = grid_dt.best_estimator_.predict(X_test)
grid_dt_pred_proba = grid_dt.best_estimator_.predict_proba(X_test)[:,1]



rf 최적 파라미터 :  {'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 2}
dt 최적 파라미터 :  {'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [144]:
print('rf 최적파라미터 적용 스코어')
score_cal(y_test,grid_rf_pred,grid_rf_pred_proba)

rf 최적파라미터 적용 스코어
accuracy : 0.6429, precision : 0.5478, recall : 0.2414, f1_score : 0.3351, roc_auc : 0.6337


In [145]:
print('dt 최적파라미터 적용 스코어')
score_cal(y_test,grid_dt_pred,grid_dt_pred_proba)

dt 최적파라미터 적용 스코어
accuracy : 0.6457, precision : 0.5474, recall : 0.2874, f1_score : 0.3769, roc_auc : 0.5979


In [ ]:
# 가장 높은 roc_auc 점수는 lr 스코어의 roc_auc : 0.6415

In [157]:
X_t

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,3500,3,3,1,3,9,13,1.0,0.789474,26
1,3501,4,4,1,9,19,90,2.0,0.369863,3
2,3502,4,3,1,0,8,101,15.0,0.083277,3
3,3503,2,2,0,31,9,5,2.0,0.000000,47
4,3504,1,1,0,19,22,3,3.0,0.125000,8
...,...,...,...,...,...,...,...,...,...,...
2477,5977,3,4,0,3,9,8,2.0,0.642857,40
2478,5978,1,1,0,17,2,1,1.0,0.000000,0
2479,5979,4,4,0,6,8,19,4.0,0.915493,18
2480,5980,3,3,0,26,8,5,4.0,0.444444,60


In [176]:
def precc(X):
    X = X.drop('cust_id',axis=1)
    X['주구매상품'] = le.fit_transform(X['주구매상품'])
    X['주구매지점'] = le.fit_transform(X['주구매지점'])
    X.환불금액.fillna(0,inplace = True)
    X['환불금액'] = np.where(X.환불금액 == 0 ,0,1)   
    X['최대구매액'] = X['최대구매액'].apply(lambda X : get_catmax(X))
    X['총구매액'] = X['총구매액'].apply(lambda X : get_catall(X))
    X['내점당구매건수'] = round(X.내점당구매건수)
    return X

X_t = pd.read_csv('./dataset/X_test.csv',encoding='CP949')
X_t1 = precc(X_t)
test_proba = lr.predict_proba(X_t1)[:,1]
X_a = X_t[['cust_id']]
X_a['gender']= test_proba
X_a[X_a.gender>0.5].count()

cust_id    525
gender     525
dtype: int64